## To begin, let's establish a general order of operations for this project:

### - Research good parameter values for a Tausworthe generator (non-code work)

### - Implement the generator.
#### (store the outputs in list format, for easy computation in later steps!)

In [66]:
def tausworthe(seed = 123456789101112, quantity = 100):
    assert type(seed) == int, 'Your seed must be an integer!'
    assert len(str(seed)) >= 15, 'Your seed must be at least 15 characters long!'
    
    seed = int('1' + str(seed)) # final seed is at least 16 characters long, or at least 50 bits
    print('seed: ', seed)
    # print('binary representation of your seed: ', bin(seed))
    gen_input = bin(seed)[2:]
    
    # Establish parameters for this version of the generator
    r = 5
    q = 50
    
    #Generate the specified number of psuedorandom bits
    gen_output = gen_input
    for bit in range(len(gen_input), len(gen_input) + quantity + 1):
        
        if gen_output[bit - r] == gen_output[bit - q]:
            gen_output = gen_output + '0'
        else:
            gen_output = gen_output + '1'
            
    gen_output = gen_output[51:]
    print('generated bits: ', len(gen_output))
    
    return gen_output
    

In [80]:
def create_prns(bits, L = 5):
    # Use the bits generated above to generate Uniform(0,1) PRNs.
    
    # Split the input into groups of L bits.
    num_groups = int(len(bits) / L)
    groups = [bits[i : i + L] for i in range(num_groups)]
    
    # Convert back to decimal; divide each group by 2 ** L to get a PRN
    PRNs = [int(group, 2) / 2 ** L for group in groups]
    
    return PRNs
    

In [81]:
create_prns(tausworthe())

seed:  1123456789101112
generated bits:  100


[0.46875,
 0.96875,
 0.9375,
 0.90625,
 0.8125,
 0.625,
 0.25,
 0.5,
 0.03125,
 0.0625,
 0.15625,
 0.34375,
 0.71875,
 0.46875,
 0.9375,
 0.875,
 0.78125,
 0.5625,
 0.15625,
 0.34375]

### - Code some general tests.
#### Runs + autocorrelation for independence, Chi-sq. for goodness of fit

### - Plot adjacent PRNs on a unit square! Sounds like a job for a loop.

### - Generate a few Nor(0, 1) RVs using the Tausworthe Uniforms.
#### Inverse Transform, or something else?